## Connecting to Docks

In [22]:
%pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client -q
%pip install python-dotenv -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from dotenv import load_dotenv

In [ ]:
# I only want read access
SCOPES = ['https://www.googleapis.com/auth/documents.readonly']

# Paths to token and credits
#Client secret json is downloaded from my Google Cloud Console
CREDS_PATH = 'auth/client_secret.json'
# Token json will be auto-created after you authorize your app the first time
TOKEN_PATH = 'auth/token.json'

def get_credentials():
    creds = None
    if os.path.exists(TOKEN_PATH):
        creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CREDS_PATH, SCOPES)
        creds = flow.run_local_server(port=0)
        with open(TOKEN_PATH, 'w') as token:
            token.write(creds.to_json())
    return creds

def read_google_doc(doc_id):
    creds = get_credentials() #Makes sure I am authenticated
    service = build('docs', 'v1', credentials=creds) #This connects to Docs API
    doc = service.documents().get(documentId=doc_id).execute() #Get the content
    
    content = ""
    for element in doc.get("body").get("content"):
        if "paragraph" in element:
            for text_run in element["paragraph"].get("elements", []):
                if "textRun" in text_run:
                    content += text_run["textRun"]["content"]
    return content

#Loads the environment variables, where I have saved the Doc Id
load_dotenv()
doc_id = os.getenv("GOOGLE_DOC_ID")

App will read the document every time I run it. 

Content could be cached and stored in a file, but with 8 pages it only took about 4 seconds, and adds additional value, that content is up to date, even if i make changes between uses of the app. Example on how to save the cached text locally is down below

In [ ]:
experience_text = read_google_doc(doc_id)

#print some of the content to show that this is working
print(experience_text[:1000])

Linkedin: https://www.linkedin.com/in/ola-graczyk/
 
1.      Work experience
1.1   UX/UI Designer
Time: October 2024  - November 2024 ( 2 months )
Company: DynamicoAI (Now Firemind)
About the company: DynamicoAI owned a product called Enhanced IQ. It is an enterprise-grade platform that makes it simple to scale generative AI across the organisation. The platform can be integrated with the most popular databases and systems and sensitive data is protected. It can be used by employees on all levels for example to search company sharepoint,  retrieve data, write structured prompts to make working faster.
About my role:
As part of my role at DynamicoAI, I was responsible for evaluating the Enhanced IQ platform with a fresh perspective, identifying potential issues, and suggesting improvements. My key responsibilities included:
-        Conducted a thorough review of the platform, documenting all observations, usability concerns, and bugs.
-        Compiled findings into a structured report

In [5]:
# Save locally
with open('cached_experience.txt', 'w', encoding='utf-8') as f:
    f.write(experience_text)

# Later, you can read from file without hitting Google API
# with open('cached_experience.txt', 'r', encoding='utf-8') as f:
#     experience_text = f.read()

## Scrape the web

In [5]:
%pip install requests beautifulsoup4 -q


Note: you may need to restart the kernel to use updated packages.


In [6]:
from web_scraper import scrape
url = input("Paste the link to job description: ")

title, job_description = scrape(url)


print(title)
print(job_description[300:600])

Paste the link to job description:  https://justjoin.it/job-offer/performance-tech-junior-ai-project-manager-solution-designer--warszawa-ai


Junior AI Project Manager (Solution Designer) 🧑‍💻
r (Solution Designer) 🧑‍💻
Performance Tech
Warszawa
Type of work
Full-time
Experience
Junior
Employment Type
B2B, Permanent
Operating mode
Hybrid
Tech stack
LLM
regular
AI
regular
RPA/IPaaS
nice to have
JavaScript/Python
nice to have
Job description
Online interview
Dołącz do naszego zespołu!
Szukam


w
AI/ML
Junior Machine Learning Engineer - e-Xperience Associate
Allegro
Warszawa
Type of work
Full-time
Experience
Junior
Employment Type
Permanent
Operating mode
Hybrid
Allegro
At Allegro, we build and maintain some of the most distributed and scalable applications in Central Europe. Work with us 


In [7]:
import re

def clean_title(title):
    title = re.sub(r'[^\w\s]', '', title)
    title = title.replace(' ', '_')
    title = ' '.join(title.split())
    return title.strip()
    

In [8]:
import os

def create_folder(title):
    cleaned_title = clean_title(title)

    folder_path = os.path.join(os.getcwd(), cleaned_title)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{cleaned_title}' created.")
    else:
        print(f"Folder '{cleaned_title}' already exists.")

    return folder_path

folder_path = create_folder(title)

Folder 'Junior_AI_Project_Manager_Solution_Designer_' already exists.


## LLMs + Langchain

### Gemini

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")

In [ ]:
%pip install --quiet langchain-core==0.1.23
%pip install --quiet langchain==0.1.1
%pip install --quiet langchain-google-genai==0.0.6
%pip install --quiet -U langchain-community==0.0.20

In [3]:
from langchain import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.chains import LLMChain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

#### Chain to extract job description

In [10]:
with open("job_prompt_template.txt", "r") as file:
   job_prompt_template = file.read()

job_prompt = PromptTemplate.from_template(job_prompt_template)

stuff_chain = (
    LLMChain(
        prompt=job_prompt, 
        llm=llm, 
        output_parser=StrOutputParser() 
    )
)

# Pass the job description as part of the chain input
job_description_result = stuff_chain.invoke({"text": job_description})['text']


# Print the result
print(job_description_result)

1. **Title of a job:** Junior AI Project Manager (Solution Designer)


2. **Role Summary Sentences:** This role involves designing, developing, and implementing AI solutions, with a focus on multi-agent systems and intelligent assistants.  The Project Manager will collaborate with development teams to integrate these solutions into web systems and optimize their performance, bridging the gap between technical AI aspects and business needs.  This involves testing, refining AI models, and identifying new application opportunities.


3. **Seniority Level and Years of Experience:** Junior level, with a minimum of 2 years of experience in AI project management or software development.


4. **Tech stack:** LLM, AI, RPA/IPaaS (nice to have), JavaScript/Python (nice to have).  Requires understanding of AI model architecture, integrations, APIs, databases, prompt engineering, and optimization of language models.


5. **Top Soft Skills / Attributes:**  Problem-solving skills, independent work et

In [11]:
file_path = os.path.join(folder_path, "job_description.txt")

with open(file_path, "w") as file:
    file.write(job_description_result)


#### Getting a CV

In [16]:
with open("cv_prompt_template.txt", "r") as file:
   cv_prompt_template = file.read()

cv_prompt = PromptTemplate.from_template(cv_prompt_template)

with open("cached_experience.txt", "r") as file:
    full_experience = file.read()

cv_chain = (
    {"experience": lambda _: full_experience, "job_description": lambda _: job_description_result}
    | cv_prompt         
    | llm                
    | StrOutputParser()  
)

raw_cv = cv_chain.invoke({})

In [20]:
def clean_markdown_output(text):
    return re.sub(r"^```(?:markdown)?\n(.*?)\n```$", r"\1", text, flags=re.DOTALL)

cv = clean_markdown_output(raw_cv)

In [22]:
warning_text = '### <span style="color:red;">Remember to check the actual CV output — models can write incorrect information!</span>'

cv = warning_text + "\n\n" + cv

In [23]:
# Save the CV text as a markdown file
file_path = os.path.join(folder_path, "generated_cv.md")

with open(file_path, "w", encoding='utf-8') as file:
    file.write(cv)